In [1]:
import pandas as pd

first_df = pd.read_csv("recipe_dataset/tf_idf_df.csv")
tmp_df = first_df

In [2]:
user_input = ["간장", "후춧가루", "밥", "아귀", "콩나물", "두부", "고춧가루", "소금", "계란", "참기름", "찹쌀"]

In [3]:
for idx in range(20):
    max_menu = ""
    max_score = 0
    max_list = []

    for menu in first_df["RECIPE_NM_KO"].drop_duplicates():
        each_score = 0
        avg = 0
        same_list = []

        target_df = first_df[first_df["RECIPE_NM_KO"] == menu].reset_index(drop = True)
        target_df_sum = target_df["SCORE"].sum()
        must_list = list(target_df[target_df["SCORE"] == 5]["Ingredient"])
        
        check_bool = True

        for must in must_list:
            chk_bool = must in user_input
            check_bool = check_bool and chk_bool
            
        if not check_bool:
            continue

        for i, compare in enumerate(target_df["Ingredient"]):
            for user_v in user_input:
                if compare == user_v:
                    each_score = each_score + target_df["SCORE"][i]
                    same_list.append(user_v)
                    break

        avg = each_score / target_df_sum

        if max_score < avg:
            max_score = avg
            max_menu = menu
            max_list = same_list
    
    print(max_menu)
    print(round(max_score * 100, 2))
    print(max_list)
    print("------------------------------------")
    
    drop_index = first_df[first_df["RECIPE_NM_KO"] == max_menu].index
    first_df = first_df.drop(drop_index).reset_index(drop=True)

새우죽
80.0
['밥', '소금', '참기름']
------------------------------------
찰밥
72.73
['소금', '찹쌀']
------------------------------------
고추장
68.42
['고춧가루', '소금', '찹쌀']
------------------------------------
두부오믈렛
65.22
['계란', '두부', '소금', '후춧가루']
------------------------------------
두부탕
65.0
['간장', '두부', '소금', '후춧가루']
------------------------------------
콩나물비빔밥
65.0
['밥', '소금', '콩나물']
------------------------------------
오믈렛
62.5
['계란', '소금', '후춧가루']
------------------------------------
중식식볶음밥
62.5
['계란', '밥', '소금', '후춧가루']
------------------------------------
두부양념조림
60.0
['간장', '두부', '참기름', '후춧가루']
------------------------------------
순두부
60.0
['소금']
------------------------------------
콩나물무침
59.09
['소금', '참기름', '콩나물', '후춧가루']
------------------------------------
아귀찜
58.97
['간장', '고춧가루', '소금', '아귀', '참기름', '찹쌀', '콩나물', '후춧가루']
------------------------------------
계란말이
58.82
['계란', '소금', '후춧가루']
------------------------------------
두부조림
58.82
['두부', '소금', '참기름']
------------------------------------
오므

In [5]:
tmp_df[tmp_df["RECIPE_NM_KO"] == "미소된장국"]

,Unnamed: 0,Recipe_ID,RECIPE_NM_KO,IRDNT_NM,TF-IDF_WEIGHT,TF-IDF Value,SCORE
5138,5138,494,미소된장국,간장,0.316233,0.105411,3
5139,5139,494,미소된장국,다시마,0.699040,0.349520,2
5140,5140,494,미소된장국,두부,1.250135,0.312534,4
5141,5141,494,미소된장국,미소된장,2.134452,0.711484,3
5142,5142,494,미소된장국,실파,0.307635,0.307635,1
5143,5143,494,미소된장국,팽이버섯,0.410120,0.410120,1
